In [1]:
%matplotlib widget
import numpy as np
import math
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import random

import matplotlib.pyplot as plt

import re

In [2]:
plikembd = open("../character_embeddings.txt")

embeddings = {}

for line in plikembd.readlines():
    exp1 = line.replace("\n", "").split(":")
    
    znak = exp1[0]
    embedding = [float(liczba) for liczba in exp1[1].replace(" ", "").replace("[", "").replace("]", "").split(",\t")[:-1]]

    
    embeddings[znak] = embedding

## Znajdowanie znaku na podstawie wybranego punktu w przestrzeni
Sieć Generatora będzie na outpucie dawać nam współrzędne punktów, które tworzą hasła. Punkty te będą niedokładne, dlatego musimy mieć funkcję, która będzie w stanie przetworzyć je na znak. Będzie się to odbywać na zasadzie znajdywania najbliżej położonego punktu.

W tym celu wykorzystujemy wzór do obliczenia odległości:
\begin{equation}
\textit{odległość = }
\sqrt{\sum_{i=0}^{m} (x_{1i}+x_{2i})^{2}}
\textit{, gdzie m to wielkość wektora znaku, a x oraz y to wektory punktów}
\end{equation}

### Dorzucamy trochę dodatków
Wiemy, że na wyjściu Generatora będziemy mieć stałą liczbę wyjść zależną jedynie od wielkości wektorów dla znaków. Chcemy jednak móc generować ciągi znaków o różnej długości. By tak zrobić należy wprowadzić do naszej przestrzeni znaków dodatkowy(pusty) znak. Wprowadzenie jego musi jednak pomijać całkowicie zależności jakie zostały już wyznaczone między znakami. Zdecydowałem się na stworzenie ogarniczenia, że jeżeli odległość od najbliższego punktu będzie większa niż jakaś ustalona, to zwracamy wtedy znak pusty. Całość przedstawia poniższa grafika. Pomarańczowe/żółte elementy pokazują wyznaczone znaki, a zielone to punkty trafione przez Generator. Dookoła zielonych punktów jest okrąg pokazujący zakres wyłapywania znaków, jeżeli najbliższy wyznaczony znak znajduje się poza przestrzenią wtedy zwracany jest znak pusty (🈳).

![https://i.imgur.com/7nDZ2aZ.png](https://i.imgur.com/7nDZ2aZ.png) 

#### Wnioski
+ Czy odległość dla zwróconego znaku powinna być uwzględniania w funkcji loss? Nie koniecznie się to nadaje, bo ta wartość zawsze jest dodania i nie wskaże nam wektora przesunięcia stepu w poszukiwaniu minimum
+ Dane wejściowe dla klasyfikatora można wzbogacić. Chodzi o dodanie pustego znaku(🈳) do hasła, tak by wypełnić jego okno, jakie jest ustalone na wyjściu Generatora i wejściu Dyskryminatora. Dodatkowo można poprzesuwać w różne strony takie hasło.

In [3]:
def get_closest_char_from_space(vectors, embeddings, odleglosc_do_pustego=5, pusty='🈳'):
    minimalna = math.inf
    znak = pusty
    
    for key in embeddings:
        suma = 0
        
        for vecid in range(len(vectors)):
            suma = suma + (embeddings[key][vecid] - vectors[vecid])**2
            
        odleglosc = math.sqrt(suma)
        
        if odleglosc < minimalna:
            minimalna = odleglosc
            znak = key
        
    if minimalna > odleglosc_do_pustego:
        znak = pusty
    else:
        minimalna = 0
        
    return znak, minimalna

In [4]:
def translate_char_to_embedding(char, embeddings, odleglosc_do_pustego=5, pusty='🈳'):
    if char in list(embeddings.keys()):
        return embeddings[char]
    else:
        if char == pusty:
            wymiarowosc = len(embeddings[list(embeddings.keys())[0]])
            
            #np.random.seed(random.randint(0, 100000))
            wartosci = np.random.rand(wymiarowosc)

            for index, element in enumerate(wartosci):
                wartosci[index] = wartosci[index] * random.randint(0,10)

            while get_closest_char_from_space(wartosci, embeddings)[1] <= odleglosc_do_pustego:
                #np.random.seed(random.randint(0, 100000))
                wartosci = np.random.rand(wymiarowosc)

                for index, element in enumerate(wartosci):
                    wartosci[index] = wartosci[index] * random.randint(0,10)
                    
            return wartosci
        else:
            return None

translate_char_to_embedding('🈳', embeddings)

array([3.90062712, 8.0872146 , 1.17390081, ..., 1.87586838, 5.36048875,
       1.3166711 ])

In [5]:
def outputsieci_na_znaki(embeddings, outputsieci, odleglosc_do_pustego=5):
    ile_skladowych = len(embeddings[list(embeddings.keys())[0]])
    
    zwrot = ""
    
    for x in range(int(len(outputsieci)/ile_skladowych)):
        zwrot = zwrot + get_closest_char_from_space(outputsieci[ile_skladowych*x:x*ile_skladowych+ile_skladowych], embeddings, odleglosc_do_pustego=odleglosc_do_pustego)[0]
    
    return zwrot

outputsieci_na_znaki(embeddings, [-0.0151,  0.0794,  0.1788,  0.1366, -0.0584,  0.0482,  0.1471,  0.0611,
        -0.0016, -0.0677,  0.0954,  0.0197, -0.1034,  0.0747,  0.1616, -0.0592,
        -0.0510,  0.0859, -0.0009,  0.0471,  0.0045, -0.1478,  0.1598, -0.0229,
         0.0614, -0.1198, -0.1260,  0.0950,  0.1093,  0.0679])

''

In [6]:
def outputsieci_na_znaki_batch(embeddings, batch, odleglosc_do_pustego=5):
    ile_skladowych = len(embeddings[list(embeddings.keys())[0]])
    
    zwr = []
    
    for outputsieci in batch:
        zwrot = ""
        for x in range(int(len(outputsieci)/ile_skladowych)):
            zwrot = zwrot + get_closest_char_from_space(outputsieci[ile_skladowych*x:x*ile_skladowych+ile_skladowych], embeddings, odleglosc_do_pustego=odleglosc_do_pustego)[0]
    
        zwr.append(zwrot)
    return zwr

In [7]:
def myloss(embeddings, batchoutput, odleglosc_do_pustego=1, aggfunc=np.max, aggbatch=np.mean):
    ile_skladowych = len(embeddings[list(embeddings.keys())[0]])
    
    ret = []
    
    for outputsieci in batchoutput:
        zwrot = []
        for x in range(int(len(outputsieci)/ile_skladowych)):
            zwrot.append(get_closest_char_from_space(outputsieci[ile_skladowych*x:x*ile_skladowych+ile_skladowych], embeddings, odleglosc_do_pustego=odleglosc_do_pustego)[1])
        
        zwrot = np.array(zwrot)
        ret.append(aggfunc(zwrot))
    return aggbatch(ret)

In [8]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        
        self.fc_in = nn.Linear(in_features=300, out_features=300)
        self.fc_h1 = nn.Linear(in_features=300, out_features=300)
        self.fc_h2 = nn.Linear(in_features=300, out_features=300)
        self.fc_h3 = nn.Linear(in_features=300, out_features=300)
        self.fc_h4 = nn.Linear(in_features=300, out_features=300)
        self.fc_h5 = nn.Linear(in_features=300, out_features=300)
        self.fc_h6 = nn.Linear(in_features=300, out_features=300)
        self.fc_h7 = nn.Linear(in_features=300, out_features=300)
        self.fc_out = nn.Linear(in_features=300, out_features=30)
        
        self.activation = nn.ReLU()
        self.activation2 = nn.PReLU()
        
    def forward(self, x):
        x = self.fc_in(x)
        # x = self.activation(x)
        
        x = self.fc_h1(x)
        # x = self.activation2(x)
        
        x = self.fc_h2(x)
        # x = self.activation(x)
        
        x = self.fc_h3(x)
        # x = self.activation(x)
        
        x = self.fc_h4(x)
        # x = self.activation(x)
        
        x = self.fc_h5(x)
        # x = self.activation(x)
        
        x = self.fc_h6(x)
        # x = self.activation(x)
        
        x = self.fc_h7(x)
        x = self.activation(x)
        
        x = self.fc_out(x)
        # x = self.activation2(x)
        
        return x

In [9]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        
        self.fc_in = nn.Linear(in_features=30, out_features=600)
        self.fc_h1 = nn.Linear(in_features=600, out_features=400)
        self.fc_h2 = nn.Linear(in_features=400, out_features=200)
        self.fc_h3 = nn.Linear(in_features=200, out_features=70)
        self.fc_out = nn.Linear(in_features=70, out_features=30)
        
        self.activation = nn.ReLU()
        self.activation2 = nn.Sigmoid()
        
    def forward(self, x):
        inp = x
        
        x = self.fc_in(x)
        x = self.activation(x)
        
        x = self.fc_h1(x)
        x = self.activation(x)
        
        x = self.fc_h2(x)
        x = self.activation(x)
        
        x = self.fc_h3(x)
        x = self.activation(x)
        
        x = self.fc_out(x)
        x = self.activation2(x)
        
        return x

In [10]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.xavier_normal(m.weight.data, 1.0, 0.002)
        #nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [11]:
plik = open("../100k.txt")

slowka = [slowo.lower().replace("\n", "") for index,slowo in enumerate(plik)]
slowa = []

print(len(slowka))

while len(slowa) < 20:
    los = random.randint(0, len(slowka) - 1)
    if slowka[los] not in slowa:
        slowa.append(slowka[los])

print(len(slowa))
print(slowa[0:2000])

slowa = slowka

100000
20
['fialek', 'baktrian', 'ostry6', 'loffee', 'orginal', 'liwka11', 'miska2003', 'pitka', '1asdfghjkl', 'julusia', 'marika5', 'katon99', 'thc', 'arkadyy', 'bogna1', 'pimpong', 'oliwek', 'danzka', 'sk07007', 'dance']


In [12]:
def get_longest_word(words):
    longest = 0
    for word in words:
        if len(word) > longest:
            longest = len(word)
    return longest

longestwrd = get_longest_word(slowa)

In [13]:
def zbuduj_slowo(word, embeddings, longestword, odleglosc_do_pustego=5):
    slowo = []
    
    for litera in word:
        slowo.append(translate_char_to_embedding(litera, embeddings, odleglosc_do_pustego=odleglosc_do_pustego, pusty='🈳'))
    
    reszta = longestword - len(word)
    
    for __ in range(reszta):
        slowo.append(translate_char_to_embedding("🈳", embeddings, odleglosc_do_pustego=odleglosc_do_pustego, pusty='🈳'))
        
    return np.array(slowo).reshape(-1)
print(zbuduj_slowo("halko", embeddings, 6).shape)

(9600,)


In [14]:
manualSeed = 1001
random.seed(manualSeed)
torch.manual_seed(manualSeed)

In [15]:
generator = Generator().to("cuda:0")
classifier = Classifier().to("cuda:0")

In [16]:
generator.apply(weights_init)
classifier.apply(weights_init)

Classifier(
  (fc_in): Linear(in_features=30, out_features=600, bias=True)
  (fc_h1): Linear(in_features=600, out_features=400, bias=True)
  (fc_h2): Linear(in_features=400, out_features=200, bias=True)
  (fc_h3): Linear(in_features=200, out_features=70, bias=True)
  (fc_out): Linear(in_features=70, out_features=30, bias=True)
  (activation): ReLU()
  (activation2): Sigmoid()
)

In [17]:
TensorRealOuts = torch.full((len(slowa), 30), 1, requires_grad=False).float().to("cuda:0")
TensorFakeOuts = torch.full((len(slowa), 30), 0, requires_grad=False).float().to("cuda:0")

TensorOuts = torch.cat((TensorRealOuts, TensorFakeOuts), 0)

print(TensorRealOuts)
print(TensorFakeOuts)

print(TensorOuts)

tensor([[1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        ...,
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.]], device='cuda:0')
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
tensor([[1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')


In [18]:
optimizerGenerator = optim.Adam(generator.parameters(), lr=1e-3)
optimizerClassifier = optim.Adam(classifier.parameters(), lr=1e-3)

print(optimizerGenerator)
print(optimizerClassifier)

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 0
)
Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 0
)


In [19]:
criterion = nn.BCELoss()

print(criterion)

BCELoss()


In [20]:
GeneratorLosses = []
ClassifierLosses = []

In [26]:
print(np.array([zbuduj_slowo(slowo, embeddings, longestwrd) for slowo in slowa])

KeyboardInterrupt: 

In [23]:
for epoch in range(1000):
    TensorRealIns = torch.tensor(np.array([zbuduj_slowo(slowo, embeddings, longestwrd) for slowo in slowa]), requires_grad=False).reshape(len(slowa), -1).float().to("cuda:0")
    TensorSeeds = torch.rand(len(slowa), 300, requires_grad=False).float().to("cuda:0")
    TensorFakeIns = generator(TensorSeeds)
    TensorIns = torch.cat((TensorRealIns, TensorFakeIns.detach()), 0)
    
    classifier.train()
    optimizerClassifier.zero_grad()
    
    y_ = classifier(TensorIns)
    lossClassifier = criterion(y_, TensorOuts)
    
    lossClassifier.backward()
    optimizerClassifier.step()
    
    
    
    generator.train()
    optimizerGenerator.zero_grad()
    
    y_ = classifier(TensorFakeIns)
    lossGenerator = criterion(y_, TensorRealOuts)# + myloss(embeddings, TensorFakeIns.cpu().detach().numpy(), odleglosc_do_pustego=4)
    
    lossGenerator.backward()
    optimizerGenerator.step()
    
    GeneratorLosses.append(lossGenerator)
    ClassifierLosses.append(lossClassifier)
    
#     if lossGenerator < 0.2:
#         break

#     faked = TensorFakeIns[0].cpu().detach().numpy()
#     xx = outputsieci_na_znaki(embeddings, faked, odleglosc_do_pustego=4)
    
#     wszystkieznakowe = 0
    
#     for znak in xx:
#         if znak in list(embeddings.keys()):
#             wszystkieznakowe = wszystkieznakowe + 1
    
#     if wszystkieznakowe == len(xx):
#         break
            
    
    if epoch % 2 == 0:
        print(lossClassifier, lossGenerator)
        #print(TensorSeeds)
        #print(TensorFakeIns[0])
        faked = TensorFakeIns[0].cpu().detach().numpy()
        print(outputsieci_na_znaki(embeddings, faked, odleglosc_do_pustego=4))

KeyboardInterrupt: 

In [ ]:
fix, ax = plt.subplots()
ax.plot([x for x in range(len(GeneratorLosses))], GeneratorLosses, ".", [x for x in range(len(GeneratorLosses))], ClassifierLosses, ".")

## Testowanie generatora

In [ ]:
with torch.no_grad():
    for __ in range(20):
        noise = torch.randn(300).to("cuda:0")
        fake = generator(noise)
        faked = fake.cpu().detach().numpy()
        print(outputsieci_na_znaki(embeddings, faked, odleglosc_do_pustego=10))

In [ ]:
noise = torch.tensor([0,0.99,0.1554,0.73766,0.1554,0,0.1515,0.1515,0.1515,0.1515,0.1515,0.1515,0.1515,0.1515,0.1515,0.1515,0.1515,0.1515,0.1515,0.1515]).float().to("cuda:0")
fake = generator(noise)
faked = fake.cpu().detach().numpy()
print(outputsieci_na_znaki(embeddings, faked, odleglosc_do_pustego=10))